### **Asistente de Investigación Inteligente utilizando el framework CrewAI, que facilita la creación de sistemas multiagente colaborativos en Python. **


# Paso 1: Preparar el Entorno
# 1.1. Instalar Dependencias

In [ ]:
!pip install crewai openai


# **Clave de API de OpenAI para utilizar los modelos de lenguaje**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


# Verificación rápida del sistema
**Pequeña prueba para chequear que todo está bien:**

In [ ]:
from crewai import Agent, Task, Crew, Process

# Definir un agente simple
agente_test = Agent(
    role='Investigador',
    goal='Buscar y explicar qué es la diabetes tipo II',
    backstory='Un agente experto en enfermedades crónicas asociadas al envejecimiento.',
    verbose=False
)

# Definir una tarea simple
tarea_test = Task(
    description='Explica brevemente qué es la diabetes y cómo se relaciona con el envjecimiento.',
    expected_output='Una breve explicación en lenguaje claro.',
    agent=agente_test
)

# Ejecutar con un Crew mínimo
equipo_test = Crew(
    agents=[agente_test],
    tasks=[tarea_test],
    process=Process.sequential,
    verbose=False
)

# Lanzar y mostrar el resultado
resultado = equipo_test.kickoff()
print("\n Resultado:\n", resultado)



✅ Resultado:
 La diabetes es una enfermedad crónica que se caracteriza por niveles altos de glucosa (azúcar) en la sangre. Existen dos tipos principales de diabetes: tipo I, que generalmente se desarrolla en la infancia o adolescencia debido a la incapacidad del cuerpo para producir insulina, y tipo II, que es más común y suele presentarse en adultos debido a la resistencia a la insulina y una producción insuficiente de esta hormona. La diabetes tipo II se asocia a menudo con factores de riesgo como la obesidad, el sedentarismo y una dieta poco saludable.

El envejecimiento también juega un papel importante en el desarrollo de la diabetes tipo II. A medida que las personas envejecen, es común que aumenten de peso y que el tejido muscular se reduzca, lo cual puede contribuir a la resistencia a la insulina. Además, el metabolismo tiende a volverse más lento y puede haber un aumento en la grasa corporal. Todo esto aumenta el riesgo de desarrollar diabetes tipo II en personas mayores, lo 

# Paso 2: Definir los Agentes
# Creamos cuatro agentes con roles específicos:

# Explorador: Busca información relevante.

# Evaluador: Filtra y evalúa la calidad de las fuentes.

# Sintetizador: Resume la información clave.

# Redactor: Redacta un informe coherente.

# Se puede usar ingeniería de prompts para optimizarlos


In [ ]:
from crewai import Agent

explorador = Agent(
    role='Explorador',
    goal='Buscar información relevante sobre el tema proporcionado',
    backstory='Especialista en búsqueda de información en diversas fuentes confiables.',
    verbose=False,
    allow_delegation=False
)

evaluador = Agent(
    role='Evaluador',
    goal='Evaluar la credibilidad y relevancia de las fuentes encontradas',
    backstory='Experto en análisis crítico de información y validación de fuentes.',
    verbose=False,
    allow_delegation=False
)

sintetizador = Agent(
    role='Sintetizador',
    goal='Resumir y organizar la información clave de las fuentes seleccionadas',
    backstory='Hábil en condensar información compleja en resúmenes claros y estructurados.',
    verbose=False,
    allow_delegation=False
)

redactor = Agent(
    role='Redactor',
    goal='Redactar un informe coherente y claro basado en los resúmenes proporcionados',
    backstory='Redactor profesional con experiencia en elaboración de informes técnicos.',
    verbose=False,
    allow_delegation=False
)


# Paso 3: Definir las Tareas
# Asignar tareas específicas a cada agente.
# Se puede usar ingeniería de prompts para optimizarlos

In [ ]:
from crewai import Task

tema = "La diabetes tipo II como enfermedad crónica asociada al envejecimiento"

tarea_explorador = Task(
    description=f'Buscar las fuentes más relevantes sobre el tema: {tema}',
    expected_output='Lista de fuentes con enlaces y breves descripciones.',
    agent=explorador
)

tarea_evaluador = Task(
    description='Evaluar la credibilidad y relevancia de las fuentes encontradas.',
    expected_output='Lista de fuentes seleccionadas con justificación.',
    agent=evaluador
)

tarea_sintetizador = Task(
    description='Resumir y organizar la información clave de las fuentes seleccionadas.',
    expected_output='Resumen estructurado con los puntos más importantes.',
    agent=sintetizador
)

tarea_redactor = Task(
    description='Redactar un informe coherente y claro basado en los resúmenes proporcionados.',
    expected_output='Informe final listo para ser presentado al usuario.',
    agent=redactor
)


# Paso 4: Crear el Equipo y Ejecutar
# Definir el equipo de trabajo y ejecutar el proceso de investigación:

In [ ]:
from crewai import Crew, Process

equipo_investigacion = Crew(
    agents=[explorador, evaluador, sintetizador, redactor],
    tasks=[tarea_explorador, tarea_evaluador, tarea_sintetizador, tarea_redactor],
    process=Process.sequential,  # Las tareas se ejecutan en orden
    verbose=False
)

resultado = equipo_investigacion.kickoff()
print(resultado)


# Paso 5: Mejorar el Agente Explorador para que realice búsquedas reales en la web usando duckduckgo-search

# 1. Instalar la librería

In [ ]:
!pip install duckduckgo-search


# 2. Crear una función de búsqueda web

In [ ]:
from duckduckgo_search import DDGS

def buscar_web_duckduckgo(query, max_results=5):
    resultados = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            resultados.append(f"{r['title']}\n{r['href']}\n{r['body']}\n")
    return "\n---\n".join(resultados)


# 3. Redefinir el Agente Explorador para usar la búsqueda
**En lugar de que el explorador le pregunte a GPT directamente, usaremos la búsqueda como entrada inicial:**

# Se puede usar ingenieria de prompts para optimizarlos

In [ ]:
tema = "La diabetes tipo II como enfermedad crónica asociada al envejecimiento"
resultados_web = buscar_web_duckduckgo(tema)

# Definimos un nuevo agente explorador que recibe esos resultados
from crewai import Agent, Task

explorador_real = Agent(
    role='Explorador Web',
    goal='Buscar información actual sobre el tema proporcionado y entregarla con referencias.',
    backstory='Especialista en búsqueda web confiable usando herramientas modernas.',
    verbose=False
)

tarea_explorador_real = Task(
    description=f"A partir de los siguientes resultados web encontrados:\n\n{resultados_web}\n\nResumí los hallazgos más relevantes.",
    expected_output='Resumen de hallazgos con referencias útiles.',
    agent=explorador_real
)


# 4. Ejecutar este agente

In [ ]:
from crewai import Crew, Process

crew_web = Crew(
    agents=[explorador_real],
    tasks=[tarea_explorador_real],
    process=Process.sequential,
    verbose=False
)

resultado_web = crew_web.kickoff()
print("\n Resultado con búsqueda web:\n")
print(resultado_web)


# 5. Probar con un Tema Específico

# Para iniciar el proceso, se proporciona un tema específico al agente explorador. Por ejemplo:

In [ ]:
tema = "La diabetes tipo II como enfermedad crónica asociada al envejecimiento"

tarea_explorador.description = f"Buscar las fuentes más relevantes sobre el tema: {tema}"


# **Paso 6: RAG**

# 1. Instalar dependencias necesarias

**Ejemplo para RAG**

In [ ]:
!pip install langchain faiss-cpu tiktoken pypdf


In [ ]:
!pip install -U langchain-community


# 2. Cargar un documento
**Archivos PDFs o TXTs, se puede subir manualmente a Colab:**

In [ ]:
from google.colab import files
uploaded = files.upload()


# 3. Crear el índice FAISS con LangChain

**Una vez subido el archivo, ejecutar el siguiente código para procesarlo y crear el índice FAISS:**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Cargar el documento PDF
loader = PyPDFLoader("La diabetes en las personas mayores.pdf")
documents = loader.load()

# Dividir el texto en fragmentos más pequeños
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Crear embeddings y construir el índice FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)


# 4. Realizar una consulta al índice

**Realizar una consulta al índice para recuperar información relevante del documento:**

In [ ]:
# Definir la consulta
query = "¿Cuáles son los síntomas de la Diabetes?"

# Realizar la búsqueda en el índice
docs = vectorstore.similarity_search(query, k=3)

# Mostrar los resultados
for i, doc in enumerate(docs):
    print(f"\nResultado {i+1}:\n")
    print(doc.page_content)
